In [1]:
import pathpyG as pp
from torch_geometric.utils import cumsum, coalesce, degree, sort_edge_index
import torch

from scipy.sparse.csgraph import bellman_ford, dijkstra
import numpy as np

from collections import defaultdict


from tqdm import tqdm

In [2]:
t_sp = pp.TemporalGraph.from_csv('sociopatterns_highschool_2013.tedges').to_undirected()
print(t_sp)
print(torch.unique(t_sp.data.t).size(0))

Temporal Graph with 327 nodes, 11636 unique edges and 377016 events in [1385982080.0, 1386345600.0]

Graph attributes
	dst		<class 'torch.Tensor'> -> torch.Size([377016])
	t		<class 'torch.Tensor'> -> torch.Size([377016])
	src		<class 'torch.Tensor'> -> torch.Size([377016])

1157


/opt/conda/lib/python3.10/site-packages/torch_geometric/data/storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'dst', 't', 'src'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


In [7]:
tedges = [('a', 'b', 1), ('b', 'c', 5), ('c', 'd', 9), ('c', 'e', 9),
              ('c', 'f', 11), ('f', 'a', 13), ('a', 'g', 18), ('b', 'f', 21),
              ('a', 'g', 26), ('c', 'f', 27), ('h', 'f', 27), ('g', 'h', 28),
              ('a', 'c', 30), ('a', 'b', 31), ('c', 'h', 32), ('f', 'h', 33),
              ('b', 'i', 42), ('i', 'b', 42), ('c', 'i', 47), ('h', 'i', 50)]
t = pp.TemporalGraph.from_edge_list(tedges)
c = pp.algorithms.centrality.temporal_closeness_centrality(t, 5)
print(c)
print(t.data.edge_index[0,:])

  0%|          | 0/17 [00:00<?, ?it/s]

100%|██████████| 17/17 [00:00<00:00, 744.39it/s]

Created temporal event DAG with 38 nodes and 47 edges
{'a': 12.0, 'b': 16.0, 'c': 16.0, 'd': 14.666666666666666, 'e': 14.666666666666666, 'f': 24.0, 'g': 14.666666666666666, 'h': 28.0, 'i': 24.0}
tensor([0, 1, 2, 2, 2, 5, 0, 1, 0, 7, 2, 6, 0, 0, 2, 5, 1, 8, 2, 7])


In [3]:
c = pp.algorithms.centrality.temporal_closeness_centrality(t_sp, 3600)

100%|██████████| 1157/1157 [01:13<00:00, 15.73it/s]


Created temporal event DAG with 377670 nodes and 31906926 edges


In [4]:
print(c)

{'454': 41671.338095238054, '640': 42220.10476190471, '1': 31384.485714285773, '939': 43385.1666666666, '185': 46485.271428571396, '258': 42662.533333333326, '55': 41131.62698412695, '170': 43822.54999999994, '9': 55537.204761904715, '453': 42306.650000000016, '45': 54390.77142857138, '14': 42915.57142857141, '190': 51695.83809523803, '400': 37502.9365079365, '637': 40505.24126984126, '255': 43516.342857142794, '275': 53779.13333333323, '176': 52638.13333333328, '533': 36436.70952380954, '116': 46335.46666666662, '151': 46517.87142857134, '866': 51019.77619047604, '280': 40957.63095238092, '484': 38709.39523809525, '243': 42567.83809523808, '687': 43355.67142857139, '54': 44738.06666666662, '364': 43477.92142857135, '374': 41239.258730158705, '295': 37942.38968253968, '441': 44693.04761904753, '101': 46261.46984126977, '425': 40808.355411255354, '47': 34439.18333333336, '241': 40734.0880952381, '179': 51910.84285714278, '202': 47549.81666666654, '63': 48038.816666666564, '564': 38769.9

In [3]:
def temporal_event_graph(g: pp.TemporalGraph, delta=1): 

    print(g.data.edge_index)

    # generate temporal event DAG
    edge_index = pp.algorithms.lift_order_temporal(g, delta)
    print(edge_index)

    # Add indices of first-order nodes as src and dst of paths in augmented
    # temporal event DAG
    src_edges_src = g.data.edge_index[0] + g.data.edge_index.size(1)
    print(src_edges_src)
    src_edges_dst = torch.arange(0, g.data.edge_index.size(1))

    dst_edges_src = torch.arange(0, g.data.edge_index.size(1))
    dst_edges_dst = g.data.edge_index[1] + g.data.edge_index.size(1) + g.N
    print(dst_edges_dst)

    # add edges from source to edges and from edges to destinations
    src_edges = torch.stack([src_edges_src, src_edges_dst])
    dst_edges = torch.stack([dst_edges_src, dst_edges_dst])
    edge_index = torch.cat([edge_index, src_edges, dst_edges], dim=1)

    # create sparse scipy matrix
    event_graph = pp.Graph.from_edge_index(edge_index) 
    return event_graph

In [4]:
eg = pp.algorithms.lift_order_temporal(t, delta=5)
print(eg)

100%|██████████| 17/17 [00:00<00:00, 4696.56it/s]

tensor([[ 0,  1,  1,  4,  5,  8, 12],
        [ 1,  2,  3,  5,  6, 11, 14]])


In [5]:
print(temporal_event_graph(t, delta=5))

tensor([[0, 1, 2, 2, 2, 5, 0, 1, 0, 7, 2, 6, 0, 0, 2, 5, 1, 8, 2, 7],
        [1, 2, 3, 4, 5, 0, 6, 5, 6, 5, 5, 7, 2, 1, 7, 7, 8, 1, 8, 8]])


100%|██████████| 17/17 [00:00<00:00, 5070.99it/s]

tensor([[ 0,  1,  1,  4,  5,  8, 12],
        [ 1,  2,  3,  5,  6, 11, 14]])
tensor([20, 21, 22, 22, 22, 25, 20, 21, 20, 27, 22, 26, 20, 20, 22, 25, 21, 28,
        22, 27])
tensor([30, 31, 32, 33, 34, 29, 35, 34, 35, 34, 34, 36, 31, 30, 36, 36, 37, 30,
        37, 37])
Directed graph with 38 nodes and 47 edges

Graph attributes
	num_nodes		<class 'int'>



In [2]:
def Dijkstra_SS_DAG(event_graph, src):
    # Note: assume that nodes are in topological order
    dist = defaultdict(np.inf)
    paths = defaultdict(list)
    # for each node u
        # for each successor v
            # dist[f_o(v)] = min(dist[f_o(u)]+1, dist[f_o(v)])
            # paths[f_o(v)].append(???)
    return dist, paths

In [10]:
def temporal_betweenness_brandes(g: pp.TemporalGraph, delta=1):

    # generate temporal event DAG
    edge_index = pp.algorithms.lift_order_temporal(g, delta)

    # Add indices of first-order nodes as src and dst of paths in augmented
    # temporal event DAG
    src_edges_src = g.data.edge_index[0,:] + g.data.edge_index.size(1)
    src_edges_dst = torch.arange(0, g.data.edge_index.size(1))

    dst_edges_src = torch.arange(0, g.data.edge_index.size(1))
    dst_edges_dst = g.data.edge_index[1,:] + 2*g.data.edge_index.size(1)

    # add edges from source to edges and from edges to destinations
    src_edges = torch.stack([src_edges_src, src_edges_dst])
    dst_edges = torch.stack([dst_edges_src, dst_edges_dst])
    edge_index = torch.cat([edge_index, src_edges, dst_edges], dim=1)

    # create sparse scipy matrix
    event_graph = pp.Graph.from_edge_index(edge_index) 

    # sources = root nodes with indeg 0 in DAG
    src_indices = torch.where(degree(edge_index[1], event_graph.N) == 0)
    print(src_indices)

    bw = defaultdict(lambda: 0)
    # for s in src
    # 

In [11]:
temporal_betweenness_brandes(t, delta=5)

100%|██████████| 17/17 [00:00<00:00, 4128.25it/s]

(tensor([20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
        38, 39]),)
